In [1]:
 pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import IPython.display as ipd filepath = "../input/urbansound8k/fold1/101415-3-0-2.wav" ipd.Audio(filepath)


In [ ]:
import librosa import librosa.display data, sample_rate = librosa.load(filepath) plt.figure(figsize=(12, 5))
librosa.display.waveshow(data, sr=sample_rate)

In [ ]:
from scipy.io import wavfile as wav
wave_sample_rate, wave_audio = wav.read(filepath)
print(wave_sample_rate)
print(wave_audio)
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))
plt.plot(wave_audio)

In [ ]:
import pandas as pd metadata = pd.read_csv('/urbansound8k/UrbanSound8K.csv') metadata.head(10)

In [ ]:
metadata['class'].value_counts()

In [ ]:
import seaborn as sns plt.figure(figsize=(10, 6)) sns.countplot(metadata['class']) plt.title("Count of
records in each class") plt.xticks(rotation="vertical") plt.show()


In [ ]:
mfccs = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40) print(mfccs.shape) print(mfccs)


In [ ]:
def features_extractor(file): #load the file (audio) audio, sample_rate = librosa.load(file_name,
res_type='kaiser_fast') #we extract mfcc mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate,
n_mfcc=40) #in order to find out scaled feature we do mean of transpose of value mfccs_scaled_features =
np.mean(mfccs_features.T,axis=0) return mfccs_scaled_features


In [ ]:
#Now we ned to extract the featured from all the audio files so we use tqdm import numpy as np from tqdm
import tqdm ### Now we iterate through every audio file and extract features ### using Mel-Frequency Cepstral
Coefficients extracted_features=[] for index_num,row in tqdm(metadata.iterrows()): file_name =
os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
final_class_labels=row["class"] data=features_extractor(file_name)
extracted_features.append([data,final_class_labels])

In [ ]:
#Now we ned to extract the featured from all the audio files so we use tqdm import numpy as np from tqdm
import tqdm ### Now we iterate through every audio file and extract features ### using Mel-Frequency Cepstral
Coefficients extracted_features=[] for index_num,row in tqdm(metadata.iterrows()): file_name =
os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
final_class_labels=row["class"] data=features_extractor(file_name)
extracted_features.append([data,final_class_labels])

In [ ]:
## Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist()) y=np.array(extracted_features_df['class'].tolist()) ###
Label Encoding -> Label Encoder from tensorflow.keras.utils import to_categorical from sklearn.preprocessing
import LabelEncoder labelencoder=LabelEncoder() y=to_categorical(labelencoder.fit_transform(y)) ### Train
Test Split from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


In [ ]:
from tensorflow.keras.models import Sequential from tensorflow.keras.layers import
Dense,Dropout,Activation,Flatten from tensorflow.keras.optimizers import Adam from sklearn import metrics ###
No of classes num_labels=y.shape[1]
model=Sequential() ###first layer model.add(Dense(100,input_shape=(40,))) model.add(Activation('relu'))
model.add(Dropout(0.5)) ###second layer model.add(Dense(200)) model.add(Activation('relu'))
model.add(Dropout(0.5)) ###third layer model.add(Dense(100)) model.add(Activation('relu'))
model.add(Dropout(0.5)) ###final layer model.add(Dense(num_labels)) model.add(Activation('softmax'))


In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')


In [ ]:
## Trianing my model from tensorflow.keras.callbacks import ModelCheckpoint from datetime import datetime
num_epochs = 100 num_batch_size = 32 checkpointer = ModelCheckpoint(filepath='./audio_classification.hdf5',
verbose=1, save_best_only=True) start = datetime.now() model.fit(X_train, y_train, batch_size=num_batch_size,
epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1) duration =
datetime.now() - start print("Training completed in time: ", duration)

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0) print(test_accuracy[1])


In [ ]:
#model.predict_classes(X_test) predict_x=model.predict(X_test) classes_x=np.argmax(predict_x,axis=1)
print(classes_x)


In [ ]:
filename="../input/urbansound8k/fold7/101848-9-0-0.wav" #preprocess the audio file audio, sample_rate =
librosa.load(filename, res_type='kaiser_fast') mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate,
n_mfcc=40) mfccs_scaled_features = np.mean(mfccs_features.T,axis=0) #Reshape MFCC feature to 2-D array
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
#predicted_label=model.predict_classes(mfccs_scaled_features) x_predict=model.predict(mfccs_scaled_features)
predicted_label=np.argmax(x_predict,axis=1) print(predicted_label) prediction_class =
labelencoder.inverse_transform(predicted_label) print(prediction_class)